# Nimbus finetuning notebook
Nimbus may not work perfectly out of the box for your dataset. This notebook will guide you through the process of finetuning Nimbus for your dataset.

In [ ]:
# import required packages
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.simplefilter("ignore")
from nimbus_inference.trainer import Trainer, AugmentationPipeline
from nimbus_inference.utils import LmdbDataset, prepare_training_data, MultiplexDataset
from nimbus_inference.nimbus import Nimbus, prep_naming_convention
from nimbus_inference.viewer_widget import NimbusViewer
from ark.utils import example_dataset
from alpineer import io_utils
import pandas as pd
import os

## 0: Set root directory and download example dataset
Here we are using the example data located in `/data/example_dataset/input_data`. To modify this notebook to run using your own data, simply change `base_dir` to point to your own sub-directory within the data folder. Set `base_dir`, the path to all of your imaging data (i.e. multiplexed images and segmentation masks). Subdirectory `training_artefacts` will contain all of the data generated by this notebook. In the following, we expect this folder structure, with `fov_1` and `fov_2` either being folders of individual channel images or `.ome.tiff` files that contain all channels in a single file.
```bash
|-- base_dir
|   |-- image_data
|   |   |-- fov_1
|   |   |-- fov_2
|   |-- segmentation
|   |   |-- deepcell_output
|   |-- training_artefacts
|-- ground_truth.csv
```

In [2]:
# set up the base directory
base_dir = os.path.normpath("../data/example_dataset")

If you would like to test Nimbus finetuning with an example dataset, run the cell below. It will download a dataset consisting of 10 FOVs with 22 channels. You may find more information about the example dataset in the [ark-analysis README](https://github.com/angelolab/ark-analysis/blob/bc6685050dfbef4607874fbbadebd4289251c173/README.md#example-dataset). If you want to use your own data, skip the cell below

In [3]:
example_dataset.get_example_dataset(dataset="cluster_pixels", save_dir = base_dir, overwrite_existing = False)

## 1: set file paths and parameters

### All data, images, files, etc. must be placed in the 'data' directory, and referenced via '../data/path_to_your_data'

The groundtruth dataframe should be a csv file with the following columns:
- `fov`: the field of view name
- `cell_id`: the unique identifier for each cell in the fov
- `channel`: the channel or marker name
- `activity`: the marker activity of the cell (`0`: negative, `1`: positive, `2`: ambiguous)

In [4]:
# set up file paths
tiff_dir = os.path.join(base_dir, "image_data")
deepcell_output_dir = os.path.join(base_dir, "segmentation", "deepcell_output")
groundtruth_path = os.path.join(base_dir, "noisy_groundtruth.csv")
training_artefacts = os.path.join(base_dir, "training_artefacts")
nimbus_output_dir = os.path.join(base_dir, "nimbus_output")

os.makedirs(training_artefacts, exist_ok=True)
os.makedirs(nimbus_output_dir, exist_ok=True)

# Check if paths exist
io_utils.validate_paths([base_dir, tiff_dir, deepcell_output_dir, training_artefacts])

# Load the ground truth data
groundtruth = pd.read_csv(groundtruth_path)

# check if the ground truth data df has column names 'fov', 'cell_id', 'channel' and 'activity'
if not set(groundtruth.columns) == set(['fov', 'cell_id', 'channel', 'activity']):
    raise ValueError(
        "Ground truth data should have columns 'fov', 'cell_id', 'channel' and 'activity'"
    )

## 2: Set up input paths and the naming convention for the segmentation data
Store names of channels to exclude in the training data in the list below. Either predict all FOVs or specify manually the ones you want to use for Nimbus training.

In [5]:
# define the channels to include
include_channels = [
    "CD3", "CD4", "CD8", "CD14", "CD20", "CD31", "CD45", "CD68", "CD163", "CK17", "Collagen1",
    "ECAD", "Fibronectin", "GLUT1", "HLADR", "IDO", "Ki67", "PD1", "SMA", "Vim"
]

# either get all fovs in the folder...
fov_names = os.listdir(tiff_dir)
# ... or optionally, select a specific set of fovs manually
# fovs = ["fov0", "fov1"]

# make sure to filter paths out that don't lead to FoVs, e.g. .DS_Store files.
fov_names = [fov_name for fov_name in fov_names if not fov_name.startswith(".")] 

# construct paths for fovs
fov_paths = [os.path.join(tiff_dir, fov_name) for fov_name in fov_names]

Define the naming convention for the segmentation data in function `segmentation_naming_convention`, that maps the `fov_name` to the path of the associated segmentation output. The below function `prep_deepcell_naming_convention` assumes that all segmentation outputs are stored in one folder, with the `fov_name` as the prefix and `_whole_cell.tiff` as the suffix, as shown below in the visualization of the folder structure. If this does not apply to your data, you have to define a function `segmentation_naming_convention` that takes an element from `fov_paths` and returns a valid path to the segmentation label map you want to use for that fov.

```
|-- base_dir
|   |-- image_data
|   |   |-- fov_1
|   |   |-- fov_2
|   |-- segmentation
|   |   |-- deepcell_output
|   |   |   |-- fov_1_whole_cell.tiff
|   |   |   |-- fov_2_whole_cell.tiff
```

In [ ]:
# Prepare segmentation naming convention that maps a fov_path to the according segmentation label map
segmentation_naming_convention = prep_naming_convention(deepcell_output_dir)

# test segmentation_naming_convention
if os.path.exists(segmentation_naming_convention(fov_paths[0])):
    print("Segmentation data exists for fov 0 and naming convention is correct")
else:
    print("Segmentation data does not exist for fov 0 or naming convention is incorrect")

Next we will use the `MultiplexDataset` class to abstract away differences in data representation. The class takes `fov_paths`, `segmentation_naming_convention` and a `suffix` and provides methods `.get_channel(fov, channel)` and `.get_segmentation(fov)` to access the data. The `suffix` is used to filter out files that do not end with the specified suffix. When you use `.ome.tiff` files make sure to set the suffix to `.ome.tiff`, otherwise the ViewerWidget won't be able to display the images.

In [ ]:
dataset = MultiplexDataset(
    fov_paths=fov_paths,
    suffix=".tiff", # or .png, .jpg, .jpeg, .tif or .ome.tiff
    include_channels=include_channels,
    segmentation_naming_convention=segmentation_naming_convention,
    output_dir=training_artefacts,
    magnification=20,
    groundtruth_df=groundtruth,
    validation_fovs=["fov8", "fov9", "fov10"], # optional: specify a list of fovs for validation
)

dataset.prepare_normalization_dict(
    quantile=0.999,
    n_subset=50,
    clip_values=(0, 2),
    multiprocessing=False,
    overwrite=True
)

## 3: Load model and initialize Nimbus application
The following code initializes the Nimbus application and loads the model checkpoint. The model was trained on a diverse set of tissues, protein markers, imaging platforms and cell types, still the performance can be improved by fine-tuning.

In [ ]:
nimbus = Nimbus(
    dataset=dataset,
    input_shape=[256,256],
    model_magnification=10,
    device="auto",  # or "cuda", "cpu", "mps" for apple silicon
    output_dir=training_artefacts,
)

# check if all inputs are valid
nimbus.check_inputs()

## 4: Prepare training data
The following code will preprocess the training data and prepare it for training. It will create a folder `training_artefacts` in the `base_dir` and store the training data in the subfolder `training_artefacts/training_data` and the validation data in subfolder `training_artefacts/validation_data` as .lmdb files. The training data consists of the images and segmentation masks of the FOVs specified in `fovs_to_train` as LMDB files. The training data is split into training and validation data with a ratio of 90:10.


In [ ]:
prepare_training_data(
        nimbus=nimbus,
        dataset=dataset,
        output_dir=training_artefacts,
        tile_size=256,
)

## 5: Setup the training pipeline and start training
The following code will setup the training pipeline, load the data and start training.

In [10]:
# load the training data
training_data = LmdbDataset(os.path.join(training_artefacts, "training"))
validation_data = LmdbDataset(os.path.join(training_artefacts, "validation"))

# define trainer
trainer = Trainer(
    nimbus=nimbus,
    train_dataset=training_data,
    validation_dataset=validation_data,
    checkpoint_name="nimbus_example_data_finetuning.pt",
    batch_size=8,
    learning_rate=1e-4,
    num_workers=1,
)

In [ ]:
trainer.train(epochs=10)

## 6: Load new model checkpoint
The following code will initialize the nimbus application and load the new model checkpoint.


In [ ]:
nimbus = Nimbus(
    dataset=dataset,
    input_shape=[512,512],
    model_magnification=10,
    device="auto",
    output_dir=nimbus_output_dir,
    checkpoint="nimbus_example_data_finetuning.pt",
    save_predictions=True,
)

## 7: Make predictions with the model
Nimbus will iterate through your samples and store predictions and a file named `nimbus_cell_table.csv` that contains the mean-per-cell predicted marker confidence scores in the sub-directory called `nimbus_output`.

In [ ]:
cell_table = nimbus.predict_fovs()

## 8: View multiplexed channels and Nimbus predictions side-by-side
Select an FOV and one marker image per channel to inspect the imaging data and associated Nimbus predictions

In [ ]:
viewer = NimbusViewer(dataset=dataset, output_dir=nimbus_output_dir)
viewer.display()